# Previsão de Preços de Frutas e Vegetais com Redes Neurais Recorrentes (RNN)

## Introdução
Neste notebook, iremos utilizar uma Rede Neural Recorrente (RNN) para prever os preços de frutas e vegetais a partir de dados históricos contidos no dataset "Fruits and Vegetables Prices Dataset". O objetivo é demonstrar como podemos aplicar técnicas de deep learning em séries temporais para capturar padrões e tendências nos preços ao longo do tempo.

## Objetivos
- Implementar uma RNN do zero para realizar previsões de preços.
- Explorar e analisar os dados históricos do dataset.
- Avaliar o desempenho do modelo e discutir possíveis melhorias.

## Bibliotecas e Configurações Iniciais
Nesta seção, serão importadas as bibliotecas necessárias (como `pandas`, `numpy`, e um framework de deep learning, por exemplo, PyTorch ou TensorFlow) e realizadas as configurações iniciais, como definição de semente para reprodutibilidade e configuração do ambiente de trabalho.


## Carregamento e Exploração do Dataset
- **Carregamento dos Dados:** Leitura do dataset (por exemplo, de um arquivo CSV) contendo os preços históricos de frutas e vegetais.
- **Exploração Inicial:** Exibição das primeiras linhas, análise de informações gerais (dimensões, tipos de dados, valores ausentes) e visualização de gráficos para entender a evolução dos preços ao longo do tempo.

In [1]:
!pip install mlcroissant

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.0/140.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.9/564.9 kB 28.8 MB/s eta 0:00:00
  Created wheel for jsonpath-rw: filename=jsonpath_rw-1.4.0-py3-none-any.whl size=15130 sha256=cd1192c735926eb5d38fc587f01095c811031adc64f77e41d74e33668ec9eaff
  Stored in directory: /root/.cache/pip/wheels/53/cf/51/a4ea10224b7fdb523e18e2033cadf2a8657517d1f95f3f5413
Successfully built jsonpath-rw


In [2]:
import mlcroissant as mlc
import pandas as pd

# Fetch the Croissant JSON-LD
croissant_dataset = mlc.Dataset('https://www.kaggle.com/datasets/everydaycodings/produce-prices-dataset/croissant/download')

# Check what record sets are in the dataset
record_sets = croissant_dataset.metadata.record_sets
print(record_sets)

# Fetch the records and put them in a DataFrame
record_set_df = pd.DataFrame(croissant_dataset.records(record_set=record_sets[0].uuid))
record_set_df.head()


  -  [Metadata(Fruits and Vegetables Prices Dataset)] Property "http://mlcommons.org/croissant/citeAs" is recommended, but does not exist.


[RecordSet(uuid="ProductPriceIndex.csv")]


,ProductPriceIndex.csv/productname,ProductPriceIndex.csv/date,ProductPriceIndex.csv/farmprice,ProductPriceIndex.csv/atlantaretail,ProductPriceIndex.csv/chicagoretail,ProductPriceIndex.csv/losangelesretail,ProductPriceIndex.csv/newyorkretail,ProductPriceIndex.csv/averagespread
0,b'Strawberries',2019-05-19,b'$1.16',b'$2.23',b'$1.70',b'$1.99',b'$2.54',b'82.33%'
1,b'Romaine Lettuce',2019-05-19,b'$0.35',b'$1.72',b'$2.00',b'$1.69',b'$1.99',b'428.57%'
2,b'Red Leaf Lettuce',2019-05-19,b'$0.32',b'$1.84',b'$1.84',b'$1.69',b'$1.89',b'467.19%'
3,b'Potatoes',2019-05-19,b'$1.50',b'$5.32',b'$5.14',b'$3.99',b'$6.22',b'244.50%'
4,b'Oranges',2019-05-19,b'$0.41',b'$1.42',b'$1.45',b'$1.34',b'$2.05',b'281.71%'


## Pré-processamento dos Dados
- **Tratamento de Datas:** Conversão da coluna de datas para o formato `datetime` e, se necessário, definição da coluna de data como índice.
- **Limpeza dos Dados:** Identificação e tratamento de valores ausentes ou inconsistentes.
- **Normalização:** Escalonamento dos preços para facilitar o treinamento do modelo.
- **Criação de Sequências Temporais:** Transformação dos dados em sequências (por exemplo, utilizando os últimos 30 dias de preços para prever o preço do dia seguinte), garantindo que a ordem temporal seja preservada.

In [3]:
!pip install matplotlib

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


df = record_set_df.copy()

columns_mapping = {
    'ProductPriceIndex.csv/productname': 'productname',
    'ProductPriceIndex.csv/date': 'date',
    'ProductPriceIndex.csv/farmprice': 'farmprice',
    'ProductPriceIndex.csv/atlantaretail': 'atlantaretail',
    'ProductPriceIndex.csv/chicagoretail': 'chicagoretail',
    'ProductPriceIndex.csv/losangelesretail': 'losangelesretail',
    'ProductPriceIndex.csv/newyorkretail': 'newyorkretail',
    'ProductPriceIndex.csv/averagespread': 'averagespread'
}
df = df.rename(columns=columns_mapping)

for col in df.columns:
    if df[col].dtype == object:
        if len(df) > 0 and isinstance(df[col].iloc[0], bytes):
            df[col] = df[col].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

df['date'] = pd.to_datetime(df['date'])
df = df.loc[:, ~df.columns.duplicated()]

In [12]:
# 2. LIMPEZA DOS DADOS
# Remover linhas sem nome de produto
df = df.dropna(subset=['productname'])

# Converter colunas de preço para númerico (remover $ e converter para float)
price_columns = ['farmprice', 'atlantaretail', 'chicagoretail',
                'losangelesretail', 'newyorkretail']

for col in price_columns:
    # Tratar strings vazias e converter para float com tratamento de erros
    df[col] = df[col].replace('', np.nan)  # Substituir strings vazias por NaN
    df[col] = df[col].str.replace('$', '', regex=False)  # Remover $
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Converter para float, forçando erros para NaN

# Verificar valores ausentes nas colunas de preço
missing_values = df[price_columns].isna().sum()
print("Valores ausentes nas colunas de preço:\n", missing_values)

# Preencher valores ausentes com a média da coluna
for col in price_columns:
    df[col] = df[col].fillna(df[col].mean())

# Recalcular a coluna averagespread
df['averagespread'] = ((df[price_columns[1:]].mean(axis=1) / df['farmprice'] - 1) * 100)



Valores ausentes nas colunas de preço:
 farmprice           1
atlantaretail       1
chicagoretail       0
losangelesretail    0
newyorkretail       8
dtype: int64


In [13]:
# 3. NORMALIZAÇÃO
# Criar versão normalizada dos dados para os preços
scaler = MinMaxScaler()
df_normalized = df.copy()
df_normalized[price_columns] = scaler.fit_transform(df[price_columns])

In [15]:
# 4. CRIAÇÃO DE SEQUÊNCIAS TEMPORAIS
# Ordenar por produto e data
df_sorted = df.sort_values(['productname', 'date'])

df_time = df.copy()
df_time.set_index('date', inplace=True)

def create_sequences(data, target_col, sequence_length=30):
    X, y = [], []
    values = data[target_col].values

    for i in range(len(values) - sequence_length):
        X.append(values[i:i+sequence_length])
        y.append(values[i+sequence_length])

    return np.array(X), np.array(y)

# Criar DataFrames individuais por produto
products = df['productname'].unique()
product_dfs = {}

for product in products:
    product_df = df[df['productname'] == product].sort_values('date')
    if len(product_df) > 30:  # Garantir que há dados suficientes para sequências
        product_dfs[product] = product_df

# Exemplos de criação de sequências para alguns produtos
sequence_data = {}
for product, product_df in list(product_dfs.items())[:3]:  # Processar apenas os 3 primeiros produtos
    if len(product_df) > 30:
        X, y = create_sequences(product_df, 'farmprice', sequence_length=30)
        sequence_data[product] = (X, y)
        print(f"Produto: {product}, Formato das sequências: X {X.shape}, y {y.shape}")

df.to_csv('produce_prices_preprocessed.csv', index=False)
df_normalized.to_csv('produce_prices_normalized.csv', index=False)
df_time.to_csv('produce_prices_time_indexed.csv')


Produto: Strawberries, Formato das sequências: X (943, 30), y (943,)
Produto: Romaine Lettuce, Formato das sequências: X (983, 30), y (983,)
Produto: Red Leaf Lettuce, Formato das sequências: X (984, 30), y (984,)


## Definição da Arquitetura da RNN
- **Modelo:** Criação de uma classe para a RNN, definindo:
  - Uma camada recorrente (pode ser uma RNN simples, LSTM ou GRU).
  - Uma camada de saída que gera a previsão do preço.
- **Funcionamento:** Explicação de como a RNN utiliza o histórico de preços para aprender os padrões temporais e realizar as previsões.

## Configuração do Treinamento
- **Preparação dos Dados:** Conversão das sequências e dos valores-alvo em tensores (caso use PyTorch, por exemplo) e criação de DataLoaders para facilitar o processamento em batches.
- **Definição da Função de Perda e Otimizador:** Uso de uma métrica de erro (como o Mean Squared Error - MSE) e escolha de um otimizador (por exemplo, Adam).
- **Configuração de Hiperparâmetros:** Definição do número de épocas, tamanho do batch, taxa de aprendizado, entre outros.

## Treinamento do Modelo
- **Loop de Treinamento:** Processamento dos batches de dados, cálculo da perda, realização do backpropagation e atualização dos pesos.
- **Monitoramento:** Registro e visualização da evolução da perda durante o treinamento para acompanhar o desempenho do modelo.

## Avaliação e Predição
- **Avaliação:** Utilização de um conjunto de teste para avaliar a performance do modelo, comparando as previsões com os valores reais.
- **Visualização dos Resultados:** Criação de gráficos que mostrem as previsões versus os valores reais para identificar se o modelo está capturando os padrões esperados.

## Conclusões e Próximos Passos
- **Resumo dos Resultados:** Discussão sobre o desempenho do modelo e os principais aprendizados obtidos durante o projeto.
- **Limitações e Melhorias:** Identificação das limitações atuais e sugestões de possíveis melhorias, como ajuste de hiperparâmetros ou experimentação com arquiteturas mais avançadas (por exemplo, LSTM ou GRU).
- **Perspectivas Futuras:** Ideias para expandir o projeto, aplicando a metodologia a outros datasets ou integrando novas fontes de dados.